In [75]:
from azureml.core.model import Model
import azureml.core

print("SDK version:", azureml.core.VERSION)

from azureml.core import Workspace
ws = Workspace.from_config()

SDK version: 1.0.85


In [76]:
model = Model.register(model_path = "./deploy_files",
                       model_name = "Gen_Model",
                       description = "Generative Model",
                       workspace = ws)

Registering model Gen_Model


In [77]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies().create(python_version='3.5')
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("torchvision")
conda_dep.add_pip_package("tqdm")
# conda_dep.add_channel('rdkit')
# conda_dep.add_channel('conda-forge')
conda_dep.add_conda_package('rdkit')
# conda_dep.add_conda_package('rdkit')
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="./score.py", environment=myenv)

In [74]:
# CREATING THE AKS TARGET - DO NOT USE AGAIN

# from azureml.core.compute import AksCompute, ComputeTarget

# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)

# aks_name = 'aks-test'
# aks_target = ComputeTarget.create(workspace = ws,
#                                      name = aks_name,
#                                      provisioning_configuration = prov_config)

# aks_target.wait_for_completion(show_output = True)

In [78]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model

aks_target = AksCompute(ws,"aks-test")

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

ERROR - Error, there is already a service with name myservice2 found in workspace ml-main



WebserviceException: WebserviceException:
	Message: Error, there is already a service with name myservice2 found in workspace ml-main
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error, there is already a service with name myservice2 found in workspace ml-main"
    }
}